In [1]:
import keras
keras.__version__

/Users/homme/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/homme/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


'2.2.2'

# 신결망과의 첫 만남

이 Notebook은 [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff)의 2장 1절에 나오는 예제 코드가 포함되어 있습니다.   
원본 텍스트([Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff))에 훨씬 더 많은 자료, 특히 추가 설명과 그림들이 포함되어 있습니다.   
여기에서는 예제 코드와 코드에 관련된 설명만 제공됩니다.

---------

자! 그럼 이제 Keras와 예제 코드를 통해 본격적으로 신경망을 배워봅시다. 우선 손글씨 숫자(hand-written)를 분류하는 문제부터 시작할 것입니다. 여러분이 앞서 Keras나 혹은 유사한 라이브러리를 한 번도 접해본 적이 없었다면, 이 첫 번째 예제를 완벽하게 이해하는 것은 당연히 불가능합니다. 아직 Keras 설치조차 하지 않은 분들이 대다수일 지도 모릅니다. 하지만 걱정하지 마세요! 지극히 정상입니다. 앞으로 여러분을 위해 예제 코드 한 줄씩 천천히 리뷰하고 아주 상세하게 설명해 드릴 것입니다. 그러니 지금 모든 것이 마법 같아 보여도 너무 걱정하지 않으셔도 됩니다! 배움을 시작하려면 어쨌든 첫걸음을 내디뎌야만 합니다!


이번 장에서 해결하고자 하는 문제는 그레이스케일 이미지(크기는 28 x 28픽셀 입니다)로 된 손글씨 숫자를 10개 카테고리(0~9)로 분류하는 것입니다.
데이터 세트는 MNIST를 사용하겠습니다. MNIST 데이터 세트는 기계학습 커뮤니티에서 아주 오랫동안 사용된 데이터 세트입니다.
기계학습 분야 그 자체만큼이나 아주 오래된 데이터 세트로서 사람들이 연구에 널리 사용하는 데이터 세트입니다.
MNIST에는 60,000개의 학습 이미지와 10,000개의 테스트 이미지가 있습니다.
이 데이터 세트는 1980년대 국립표준기술연구소(National Institute of Standards and Technology, NIST in MNIST) 에서 수집하였습니다.
MNIST 분류 문제를 딥러닝의 "Hello world" 격으로 보시면 됩니다.
여러분은 MNIST로 여러분이 만든 알고리즘이 예상처럼 잘 동작하는지 검증하는 목적으로 사용할 수 있습니다.
딥러닝 쪽으로 연구를 계속 진행하다 보면, 여러 논문, 블로그 등에서 MNIST 데이터 세트를 수도 없이 볼 수 있을 것입니다.

MNIST 데이터 세트를 Keras에서 로드합니다. MNIST는 Numpy arrays의 형태로 로드됩니다.

In [2]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

`train_images` 과 `train_labels` 은 "학습 세트" 입니다. 이 데이터로 모델을 학습시킬 것입니다.  
그리고 모델의 테스트는 "테스트 세트"를 이용합니다, `test_images` 과 `test_labels`입니다.  
MNIST 데이터에서 이미지는 Numpy arrays로 인코딩되어 있습니다. 그리고 레이블은 0에서 9 사이의 숫자로 인코딩되어 있습니다.   
학습 집합에서는 각 이미지와 정답 레이블이 쌍으로 존재합니다

우선 학습 세트을 먼저 살펴봅시다.

In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
len(train_labels)

60000

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

그럼 이제 테스트 세트를 살펴봅시다:

In [6]:
test_images.shape

(10000, 28, 28)

In [7]:
len(test_labels)

10000

In [8]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

작업수행 과정은 다음과 같습니다: 우선 네트워크를 학습 데이터(`train_images` 와 `train_labels`)로 학습시킵니다. 네트워크가 학습 데이터의 이미지와 레이블을 학습할 것입니다. 그리고 학습시킨 모델로 `test_images`를 예측해 볼 것입니다. 그리고 예측한 값을 `test_labels`로 검증합니다.   

이제 네트워크를 구성해봅시다! 다시 한번 말씀드리지만, 지금 당장 이 모든 과정을 다 이해하지 못해도 좋습니다! 

In [9]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

신경망을 구성할 때 가장 중요한 요소는 "레이어" 입니다. "레이어"는 데이터 처리 모듈로 "필터"라고 생각하시면 됩니다. 데이터는 "레이어"의 입력으로 들어가서, 더 유용한 형태로 반환됩니다. 조금 더 자세히 말씀드리면, 레이어들은 입력으로 들어온 데이터에서 representations를 추출하는 역할을 합니다. 추출한 representations 우리 문제를 풀기에 더 의미 있는 특징들을 가지고 있길 바라면서 말이죠. 대부분의 딥러닝 모델들은 단순한 레이어들을 한데 엮어서 구성합니다. 여러 레이어들을 거치면서 점진적으로 "데이터 증류(data distillation)"의 임무를 수행할 것입니다. 딥러닝은 "레이어"라는 `데이터 정재 필터`를 이용해서 데이터를 점진적으로 채로 걸러내는 역할을 한다고 볼 수 있습니다. 

여기에서 우리가 만들 네트워크는 두 개의 `Dense` 레이어로 구성됩니다. `Dense` 레이어는 "fully connected(densely-connected) layer" 라고도 합니다. 두 번째(마지막) 레이어는 10-way "softmax" 레이어입니다. 이 레이어는 10가지 확률 스코어를 array의 형태로 반환합니다. (모든 스코어를 더하면 1이 됩니다) 10개의 각 점수는 현재 이미지가 10개의 숫자 클래스 중 하나에 속할 확률입니다.

아직 끝이 아닙니다! 네트워크를 학습시키기에 앞서, 세 가지 작업이 더 필요합니다. 구성한 모델을 컴파일하는 "compilation" 스텝이 필요합니다.

* 손실 함수 : 손실함수는 학습 세트에 대해서 네트워크가 얼마나 잘 학습되었는지, 올바른 방향으로 학습되었는지에 대한 지표입니다.
* 옵티마이저 : 옵티마이저는 네트워크가 학습 데이터와 손실함수를 기반으로 모델을 업데이트하기 위한 메커니즘입니다. 
* 학습/시험 시 모니터링을 위한 측정항목: 이번 예제에서는 정확도(Accuracy) 만 계산하겠습니다. (정확도는 모델이 이미지를 얼마나 잘 분류하였는지 대한 비율입니다.)

손실 함수와 옵티마이저에 대해서는 다음 두 장에 걸쳐서 더 자세히 배워보도록 하겠습니다. 

In [10]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

본격적인 학습에 앞서 데이터 전처리가 필요합니다. 데이터의 shape도 네트워크가 요구하는 shape로 적절히 변형시켜야 하며, 값의 범위도 `[0, 1]` 로 맞춰야 합니다. 앞서 우리가 가지고 있는 학습 이미지의 shape는 `(60000, 28, 28)`, type은 `uint8`, 값이 범위는 `[0, 255]` 이었습니다. 이제 학습 이미지를 `(60000, 28 * 28)`, `float32`, 그리고 `[0, 1]`로 변환시켜 주겠습니다.

In [11]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

레이블 데이터는 범주형으로 인코딩해야 합니다. 이와 관련된 내용은 3장에서 다시 설명해 드리겠습니다.

In [12]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

자 이제 네트워크를 학습시킬 준비가 끝났습니다. 이제 Keras에서 `fit` 함수를 호출하기만 하면 됩니다. : 이제 모델을 학습 데이터에 "fit" 할 것입니다. 

In [13]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 2s 38us/step - loss: 0.2628 - acc: 0.9244
Epoch 2/5
60000/60000 [==============================] - 2s 35us/step - loss: 0.1048 - acc: 0.9690
Epoch 3/5
60000/60000 [==============================] - 2s 35us/step - loss: 0.0688 - acc: 0.9787
Epoch 4/5
60000/60000 [==============================] - 2s 36us/step - loss: 0.0500 - acc: 0.9853
Epoch 5/5
60000/60000 [==============================] - 2s 35us/step - loss: 0.0375 - acc: 0.9888


학습 동안에 두 가지 수치가 출력됩니다. 하나는 학습 데이터에 대한 네트워크의 "손실" 입니다. 그리고 다른 하나는 학습 데이터에 대한 네트워크의 "정확도" 입니다. 

학습 데이터로는 0.989의 정확도를 달성했습니다. 자 이제 시험 세트의 성능을 살펴봅시다. 

In [14]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

10000/10000 [==============================] - 0s 29us/step


In [15]:
print('test_acc:', test_acc)

test_acc: 0.9778


테스트 정확도는 97.8%를 기록했습니다 -- 이 결과는 사실 학습 세트의 정확도보다는 조금 낮습니다. 학습 정확도보다 테스트 정확도가 낮은 이유는 "과적합(overfilling)"과 관련이 있습니다. 기계학습 모델들은 학습 데이터에서의 성능보다 새로운 데이터에서 성능이 더 나빠지는 경향이 있습니다. 과적합 문제는 3장에서 자세히 다루도록 하겠습니다.

지금까지 첫 번째 예시를 살펴보았습니다. 결론은 다음과 같습니다. 이번 장에서는 20줄조차 안되는 간단한 Python 코드로 손글씨 숫자를 분류하는 신경망을 구축하고 실제로 학습시켜보았습니다. 다음 장에서는 우리가 지금까지 살펴봤던 구성요소들을 조금 더 자세히 다뤄보겠습니다. 다음 시간에는 네트워크 내부에서 데이터를 저장하는 객체인 "tensors"와, tensors가 실제로 만들어지는 tensor operations, 그리고 네트워크가 학습 데이터로 학습을 하게 해주는 gradient descent에 대해서 배워볼 것입니다.